### Basic code to read easterly wave tracks by Lawton et al (2022) and plot a track that matches user specified tropical storm (if available in the track database)


- Lawton, Q. A., S. J. Majumdar, K. Dotterer, C. Thorncroft, and C. J. Schreck, 2022: The Influence of Convectively Coupled Kelvin Waves on African Easterly Waves in a Wave-Following Framework. Monthly Weather Review, 150(8), 2055-2072, https://doi.org/10.1175/MWR-D-21-0321.1.


- NCSU Tropical and Large Scale Dynamics
- A. Aiyyer

In [ ]:
import numpy as np
import xarray as xr
from datetime import date, timedelta, datetime
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.animation as animation
# convert the animation to javascript for display
#plt.rcParams["animation.html"] = "jshtml"
import geocat.viz.util as gvutil
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
plt.rcParams['animation.embed_limit'] = 2**128
from metpy.plots import colortables

In [ ]:
# get the easterly wave track data from https://zenodo.org/records/13350860

# location of the data on the local machine:
track_data_path = "/Users/bmapes/GitHub/SkySymphony/ERA5_AEW_TRACKS/"
file_prefix = "AEW_tracks_post_processed_year_"

In [ ]:
# Select storm 
storm_Name = "DOLLY"
storm_Year = 2008

# Select storm
storm_Name = "IRMA"
storm_Year = 2017

# Select storm
storm_Name = "ERNESTO"
storm_Year = 2012


In [ ]:
fileName = track_data_path +  file_prefix + str(storm_Year) + '.nc'
ds = xr.open_dataset(fileName)

In [ ]:
lon = ds.AEW_lon
lat = ds.AEW_lat
n_waves = lon.shape[0]
print(n_waves)

In [ ]:
idx = np.where(ds.TC_name == storm_Name)[0]

if idx.shape == (0,):
   print ("No storm match found")
else:
    istorm = idx[0]
    print(istorm, storm_Name)

In [ ]:
#locate a specific TC
lon_s = ds.AEW_lon.isel(system=istorm).dropna(dim='time')
lat_s =  ds.AEW_lat.isel(system=istorm).dropna(dim='time')
TC_genesis_time = ds.TC_gen_time.isel(system=istorm)
#locate the track index where the wave becomes a TC
idt = np.where(lat_s.time == TC_genesis_time)[0][0]
print("Storm genesis at ", idt, TC_genesis_time.values)

In [ ]:
# PLOT the track of the wave + TC

In [ ]:
fig = plt.figure(figsize=(10, 6))
proj = ccrs.PlateCarree()  # cylindrical equidistant
ax = fig.add_subplot(1, 1, 1, projection=proj)
# Add coastlines and gridlines
ax.coastlines()
#ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
# Plot  wave track
ax.plot(lon_s, lat_s, marker='none', markersize=2, transform=ccrs.PlateCarree())
ax.plot(lon_s[idt:], lat_s[idt:], marker='o', markersize=4, transform=ccrs.PlateCarree())
plt.title(storm_Name + ": Wave Track", fontsize=14)
plt.show()

In [ ]:
# now download and animate gridsat ir images

In [ ]:
dates=lon_s.time.values
date_storm = pd.to_datetime(dates[0])

In [ ]:
# set the min and max values to shade in the IR brightness temperature field
Cmin=  170
Cmax = 300
Cint = 10

#------------------------------------------------
# ATL and EPAC
min_lon = -140.
max_lon =   30.
delta_lon = 20.
min_lat = -10.
max_lat =  45.
#--------------------------------------------
central_lon = .5*(min_lon + max_lon)
ntimes = lon_s.size

In [ ]:
def animate(i,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax):
    central_lon = .5*(min_lon + max_lon)
    delta_lon = 20.
    #datePlot = date_storm + pd.Timedelta(days=i*.25)
    datePlot = pd.to_datetime(dates[i])
    Y = datePlot.strftime('%Y')
    # if accessing from ncei
    url_base = 'https://www.ncei.noaa.gov/thredds/dodsC/cdr/gridsat/' + Y + '/'
    # if locally downloaded
    #url_base = '../data/dolly2008/'
   
    url = url_base + "GRIDSAT-B1."  + datePlot.strftime('%Y.%m.%d.%H')  + ".v02r01.nc"
    cmap = colortables.get_colortable('ir_tv1')
    print(" here ", i , url )
    ds = xr.open_dataset(url)
    LatIndexer, LonIndexer = 'lat', 'lon'
    irwin = ds.irwin_cdr[0,:,:].sel(**{LatIndexer: slice(min_lat, max_lat),
                        LonIndexer: slice(min_lon, max_lon)})
    print(irwin.min().values)
    ct=plt.imshow(irwin, cmap=cmap, origin='lower', vmin=Cmin,vmax=Cmax,extent=[min_lon, max_lon,min_lat, max_lat])
 
    gvutil.set_titles_and_labels(ax,
                                 lefttitle=datePlot.strftime('%Y-%H-%d %H UTC'),
                                 righttitlefontsize=12,
                                 righttitle="11 micron BT (K)",
                                 lefttitlefontsize=12,
                                 xlabel="",
                                 ylabel="")
    #ct = plt.pcolormesh(lons, lats, irwin[:-1,:-1],cmap='Greys', rasterized=True, vmin=Cmin, vmax =Cmax, shading='flat')
    plt.title(datePlot, y=1.2)
    

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

transform = ccrs.PlateCarree()

#fig = plt.figure(figsize=(18,6))
fig = plt.figure(figsize=(10,5))

# Generate axes, using Cartopy, drawing coastlines, and adding features
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.coastlines(linewidths=0.75, color='black')

i=0
Y = date_storm.strftime('%Y')
url_base = 'https://www.ncei.noaa.gov/thredds/dodsC/cdr/gridsat/' + Y + '/'
#url_base = '../data/dolly2008/'
url = url_base + "GRIDSAT-B1."  + date_storm.strftime('%Y.%m.%d.%H')  + ".v02r01.nc"
cmap = colortables.get_colortable('ir_tv1')
print(" here ", i , url )
ds = xr.open_dataset(url)

LatIndexer, LonIndexer = 'lat', 'lon'
irwin = ds.irwin_cdr[0,:,:].sel(**{LatIndexer: slice(min_lat, max_lat),
                    LonIndexer: slice(min_lon, max_lon)})
ct=plt.imshow(irwin[:-1,:-1], cmap=cmap, origin='lower', vmin=Cmin,vmax=Cmax,extent=[min_lon, max_lon,min_lat, max_lat])

print(irwin.min().values)
# Add color bar
cbar_ticks = np.arange(Cmin,Cmax,Cint)
cbar = plt.colorbar(ct,
                    orientation='horizontal',
                    shrink=0.95 ,aspect=30,
                    pad=0.15,
                    extendrect=True,
                    ticks=cbar_ticks)

cbar.ax.tick_params(labelsize=10)  

# Use geocat.viz.util convenience function to set axes limit
gvutil.set_axes_limits_and_ticks(ax,
                                xlim=(min_lon, max_lon),
                                ylim=(min_lat, max_lat),
                                xticks=np.linspace(min_lon,max_lon, 5),
                                yticks=np.linspace(min_lat,max_lat, 5))

# Use geocat.viz.util convenience function to add major tick lines
gvutil.add_major_minor_ticks(ax, y_minor_per_major=1, labelsize=12)

# Use geocat.viz.util convenience function to add lat and lon tick labels
gvutil.add_lat_lon_ticklabels(ax)

# Remove degree symbol from tick label
ax.yaxis.set_major_formatter(LatitudeFormatter(degree_symbol=''))
ax.xaxis.set_major_formatter(LongitudeFormatter(degree_symbol=''))
# Use geocat.viz.util convenience function to set titles and labels


#---------------------------------------------------------------------------------------------------
animArgs = (min_lon,max_lon,min_lat,max_lat,Cmin,Cmax)
# runs the animation initiated with the frame from init and progressed with the animate function
anim = animation.FuncAnimation(fig,animate,frames=ntimes, interval=200, fargs=animArgs)





anim    

In [ ]:
anim